In [8]:
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('Processed_data.csv')
df.head()

,STATE,IBGE_RES_POP,IBGE_RES_POP_BRAS,IBGE_DU,IBGE_DU_URBAN,IBGE_DU_RURAL,IBGE_POP,IBGE_1,IBGE_1-4,IBGE_5-9,...,Pu_Agencies,Pr_Bank,Pu_Bank,Cars,Motorcycles,Wheeled_tractor,UBER,MAC,WAL-MART,POST_OFFICES
0,16,5098,5098,1536,1100,436,3594,46,198,265,...,0,0,0,282,1185,0,0,0,0,1
1,10,2709,2706,875,364,511,1099,13,40,69,...,0,0,0,479,332,0,0,0,0,1
2,14,5239,5239,1567,639,928,2062,40,132,162,...,0,0,0,297,274,0,0,0,0,1
3,14,2366,2366,676,353,323,1231,22,113,119,...,0,0,0,198,104,0,0,0,0,1
4,9,11063,11063,2632,1118,1515,3154,31,204,316,...,0,0,0,196,1008,0,0,0,0,1


In [9]:
# Split the data into features and target
X = df.drop(columns=['IDHM'])
y = df['IDHM']

In [11]:
import numpy as np

# Verificar valores faltantes em X_train e X_test
print("Valores faltantes em X_train:")
print(X_train.isnull().sum())

print("Valores faltantes em X_test:")
print(X_test.isnull().sum())

# Tratar valores infinitos e NaNs em X_train e X_test
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)

# Preencher valores NaN com a média das colunas
X_train.fillna(X_train.mean(), inplace=True)
X_test.fillna(X_test.mean(), inplace=True)

Valores faltantes em X_train:
STATE                0
IBGE_RES_POP         0
IBGE_RES_POP_BRAS    0
IBGE_DU              0
IBGE_DU_URBAN        0
                    ..
Wheeled_tractor      0
UBER                 0
MAC                  0
WAL-MART             0
POST_OFFICES         0
Length: 64, dtype: int64
Valores faltantes em X_test:
STATE                0
IBGE_RES_POP         0
IBGE_RES_POP_BRAS    0
IBGE_DU              0
IBGE_DU_URBAN        0
                    ..
Wheeled_tractor      0
UBER                 0
MAC                  0
WAL-MART             0
POST_OFFICES         0
Length: 64, dtype: int64


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Split dados

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalização de Escalas 

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

# Definir o grid de hiperparâmetros para Random Forest
param_grid_rf = {
    'n_estimators': [45, 50, 57],  # Número de árvores na floresta
    'max_depth': [10, 20, 30, None],  # Profundidade máxima de cada árvore
    'min_samples_split': [2, 10, 20],  # Número mínimo de amostras para dividir um nó
    'min_samples_leaf': [1, 5, 10],  # Número mínimo de amostras em cada folha
    'max_features': ['sqrt', 'log2', None],  # Número de recursos a serem considerados para a melhor divisão
    'random_state': [66]  # Garantir reprodutibilidade
}

# Configurar o GridSearch para RandomForest
grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid_rf, cv=5, scoring='neg_mean_squared_error')

# Treinar e buscar os melhores hiperparâmetros
grid_search_rf.fit(X_train, y_train)

# Exibir os melhores hiperparâmetros
print("Melhores hiperparâmetros para Random Forest:", grid_search_rf.best_params_)



Melhores hiperparâmetros para Random Forest: {'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50, 'random_state': 66}


c:\Users\mikae\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


In [14]:
# Treinar o modelo final com os melhores hiperparâmetros
final_rf_model = RandomForestRegressor(
    n_estimators=grid_search_rf.best_params_['n_estimators'],
    max_depth=grid_search_rf.best_params_['max_depth'],
    min_samples_split=grid_search_rf.best_params_['min_samples_split'],
    min_samples_leaf=grid_search_rf.best_params_['min_samples_leaf'],
    max_features=grid_search_rf.best_params_['max_features'],
    random_state=grid_search_rf.best_params_['random_state']
)

# Ajustar o modelo final com os dados de treino
final_rf_model.fit(X_train, y_train)

# Fazer previsões com o conjunto de teste
y_pred_rf = final_rf_model.predict(X_test)

# Avaliar o desempenho do modelo usando RMSE e R²
rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"RMSE do modelo Random Forest: {rmse_rf}")
print(f"R² do modelo Random Forest: {r2_rf}")

RMSE do modelo Random Forest: 0.0244419763417405
R² do modelo Random Forest: 0.7112153860174009


c:\Users\mikae\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
